# Exercise 2: Bumeran — Web Scraping

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

### Calling Libraries

In [2]:
# install the libraries
!pip install selenium 
!pip install selenium webdriver_manager pandas
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\KikeMa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\KikeMa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pwd #se identifica la carpeta donde se esta trabajando

'c:\\Users\\KikeMa\\Documents\\Azu\\diplomado\\python\\Ejercicios clase\\web_scrapping\\code'

In [4]:
### ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()
url = 'https://www.bumeran.com.pe'
driver.get(url)

## 2. Task Description

In [5]:
def setup_driver():
    """Configura y retorna un nuevo driver de Chrome"""
    try:
        service = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        return driver
    except Exception as e:
        print(f"Error configurando el driver: {e}")
        return None

def aplicar_filtros_automaticos(driver):
    """Aplica todos los filtros automáticamente desde la página principal"""
    try:
        # Ir a la página principal de búsqueda
        driver.get('https://www.bumeran.com.pe/empleos/')
        time.sleep(3)
        
        print("Aplicando filtros automáticamente...")
        
        # 1. Click en el botón de búsqueda avanzada (si existe)
        try:
            avanzada_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Búsqueda avanzada') or contains(text(), 'Avanzada')]"))
            )
            avanzada_btn.click()
            time.sleep(2)
        except:
            print("No se encontró botón de búsqueda avanzada, continuando...")
        
        # 2. Filtrar por Área: Tecnología, Sistemas y Telecomunicaciones
        try:
            area_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//select[contains(@name, 'area') or contains(@id, 'area')]"))
            )
            area_dropdown.click()
            time.sleep(1)
            
            # Seleccionar Tecnología, Sistemas y Telecomunicaciones
            tecnologia_option = driver.find_element(By.XPATH, "//option[contains(text(), 'Tecnología') or contains(text(), 'Tecnologia')]")
            tecnologia_option.click()
            time.sleep(2)
            print("Área seleccionada: Tecnología")
        except Exception as e:
            print(f"No se pudo seleccionar área: {e}")
            return False

        # 3. Filtrar por Subárea: Programación
        try:
            subarea_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//select[contains(@name, 'subarea') or contains(@id, 'subarea')]"))
            )
            subarea_dropdown.click()
            time.sleep(1)
            
            programacion_option = driver.find_element(By.XPATH, "//option[contains(text(), 'Programación') or contains(text(), 'Programacion')]")
            programacion_option.click()
            time.sleep(2)
            print("Subárea seleccionada: Programación")
        except Exception as e:
            print(f"No se pudo seleccionar subárea: {e}")
            return False

        # 4. Filtrar por Modalidad: Full-time
        try:
            modalidad_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//select[contains(@name, 'workday') or contains(@id, 'workday')]"))
            )
            modalidad_dropdown.click()
            time.sleep(1)
            
            fulltime_option = driver.find_element(By.XPATH, "//option[contains(text(), 'Full-time') or contains(text(), 'Tiempo completo')]")
            fulltime_option.click()
            time.sleep(2)
            print("Modalidad seleccionada: Full-time")
        except Exception as e:
            print(f"No se pudo seleccionar modalidad: {e}")
            return False

        # 5. Filtrar por Ubicación: Lima
        try:
            ubicacion_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//input[contains(@name, 'location') or contains(@id, 'location')]"))
            )
            ubicacion_input.clear()
            ubicacion_input.send_keys("Lima")
            time.sleep(2)
            print("Ubicación ingresada: Lima")
        except Exception as e:
            print(f"No se pudo ingresar ubicación: {e}")
            return False

        # 6. Filtrar por Fecha: Menor a 15 días
        try:
            fecha_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//select[contains(@name, 'date') or contains(@id, 'date')]"))
            )
            fecha_dropdown.click()
            time.sleep(1)
            
            fecha_option = driver.find_element(By.XPATH, "//option[contains(text(), '15 días') or contains(text(), '15 dias')]")
            fecha_option.click()
            time.sleep(2)
            print("Fecha seleccionada: Menor a 15 días")
        except Exception as e:
            print(f"No se pudo seleccionar fecha: {e}")
            return False

        # 7. Hacer clic en Buscar
        try:
            buscar_btn = driver.find_element(By.XPATH, "//button[contains(text(), 'Buscar') or contains(text(), 'Search')]")
            buscar_btn.click()
            time.sleep(5)
            print("Búsqueda ejecutada")
            return True
        except Exception as e:
            print(f"No se pudo hacer clic en Buscar: {e}")
            return False

    except Exception as e:
        print(f"Error general aplicando filtros: {e}")
        return False

## Suggested Scraping Strategy (Two Stages)
### Desarrollamos Stage 1
Scrape all the job listing URLs based on the filters above.
Navigate across all pages if necessary.

In [7]:
def scrape_all_pages(usar_filtros_automaticos=False):
    """Scrapea todas las páginas, con opción de aplicar filtros automáticamente"""
    driver = setup_driver()
    all_job_links = []
    
    if not driver:
        return []
    
    try:
        if usar_filtros_automaticos:
            # Aplicar filtros automáticamente
            if aplicar_filtros_automaticos(driver):
                base_url = driver.current_url
                print(f" URL con filtros aplicados: {base_url}")
            else:
                print(" Usando URL predefinida por fallo en filtros automáticos")
                base_url = 'https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html'
        else:
            # Usar URL predefinida
            base_url = 'https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html'
            driver.get(base_url)
            time.sleep(5)
        
        # Continuar con el scraping de páginas
        for page_num in range(1, 6):  # Páginas 1 a 5
            if page_num == 1:
                url = base_url
            else:
                # Construir URL de página
                if '?' in base_url:
                    url = f"{base_url}&page={page_num}"
                else:
                    url = f"{base_url}?page={page_num}"
            
            print(f"Accediendo a página {page_num}: {url}")
            driver.get(url)
            time.sleep(5)
            
            print(f"Título página {page_num}: {driver.title}")
            
            # Buscar todos los enlaces
            elements = driver.find_elements(By.TAG_NAME, "a")
            print(f" Total de enlaces en página {page_num}: {len(elements)}")
            
            # Filtrar enlaces de trabajos
            page_links = []
            for element in elements:
                try:
                    href = element.get_attribute("href")
                    if href and '/empleos/' in href and '.html' in href:
                        if href not in page_links and href not in all_job_links:
                            page_links.append(href)
                except StaleElementReferenceException:
                    continue
            
            print(f" Página {page_num}: {len(page_links)} trabajos encontrados")
            all_job_links.extend(page_links)
            
            # Mostrar algunos enlaces
            if page_links:
                for i, link in enumerate(page_links[:3], 1):
                    print(f"   {i}. {link}")
            else:
                print("No se encontraron trabajos en esta página")
                break
            
            time.sleep(2)
            
    except Exception as e:
        print(f" Error: {e}")
        driver.save_screenshot('error_scraping.png')
    finally:
        driver.quit()
    
    return list(set(all_job_links))

# EJECUCIÓN PRINCIPAL
print("Iniciando scraping de Bumeran")
print("=" * 50)

# Opción: True para aplicar filtros automáticamente, False para usar URL directa
usar_filtros_auto = False  # Cambia a True si quieres que aplique filtros automáticos

print(f"Modo: {'Filtros automáticos' if usar_filtros_auto else 'URL predefinida'}")

all_job_links = scrape_all_pages(usar_filtros_automaticos=usar_filtros_auto)

print(f"\n RESULTADOS:")
print(f"Total de ofertas únicas encontradas: {len(all_job_links)}")
print(f"Esperado: 91 trabajos")

# Guardar en CSV
if all_job_links:
    df = pd.DataFrame(all_job_links, columns=['job_url'])
    df.to_csv('bumeran_complete_job_links.csv', index=False, encoding='utf-8')
    print("Enlaces guardados en 'bumeran_complete_job_links.csv'")
    
    # Mostrar estadísticas
    print("\n Primeros 10 enlaces:")
    for i, link in enumerate(all_job_links[:10], 1):
        print(f"{i:2d}. {link}")
    
    # Verificar resultados
    if len(all_job_links) >= 91:
        print(" ¡Éxito! Se encontraron todos los 91 trabajos")
    elif len(all_job_links) > 0:
        print(f"Se encontraron {len(all_job_links)} de 91 trabajos")
    else:
        print("No se encontraron enlaces")

print("\nProceso completado!")

Iniciando scraping de Bumeran
Modo: URL predefinida
Accediendo a página 1: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html
Título página 1: Trabajos en Perú - Empleos Bumeran 2020
 Total de enlaces en página 1: 77
 Página 1: 20 trabajos encontrados
   1. https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
   2. https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
   3. https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html
Accediendo a página 2: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html?page=2
Título página 2: Empleos: Programación Full-time en Lima - Página 2 | Bumeran
 Total de enlaces en página 2: 84
 Página 2: 20 trabajos encontrados
   1. https://www.bumeran.com.